## [132- Palindrome Partitioning II](https://leetcode.com/problems/palindrome-partitioning-ii/)

问题难度： &clubs; &clubs; &clubs; &clubs; &clubs;

### 问题难度

给定一个字符串`s`， 对`s`进行分割使得每个子串部分都是回文。

返回`s`的回文分割中需要的最小切割次数。

**示例**
```
Input: "aab"
Output: 1
解释: 只切割1次就能形成： ["aa","b"]
```

### 解题思路

还是一句老话：所有字符串相关的题目都能用动态规划法进行求解。

数学化： 使用`cuts[i]`表示从`i`到`len(s)`的最小切割数,假设`i`到`j`之间为回文，那么`cuts[i] = min(cuts[i], cuts[j+1] + 1)`；初始化：`cuts[i]=len(s)-i-1`，最后返回`cuts[0]`。

问题变成了如何判断`i`到`j`是否为回文：
- 如果`j-i<=1`，那么只需要判断`s[i] == s[j]`
- 否则， 只需要判断`s[i]==s[j] && p[i-1][j+1]`

### 代码

In [1]:
class Solution(object):
    def minCut(self, s):
        """
        :type s: str
        :rtype: int
        """
        s_len = len(s)
        cuts = []
        p = [[False for _ in range(s_len)] for _ in range(s_len)]
        for i in range(s_len + 1):
            cuts.append(s_len-i- 1)
        for i in range(s_len-1, -1, -1):
            for j in range(i, s_len):
                if (s[i]==s[j] and (j-i<=1 or p[i+1][j-1])):
                    p[i][j] = True
                    if cuts[j+1] + 1 < cuts[i]:
                        cuts[i] = cuts[j+1] + 1
        return cuts[0]
        

## [133- Clone Graph](https://leetcode.com/problems/clone-graph/)

问题难度： &clubs; &clubs; &clubs;

### 问题难度

给定一个图的头，返回图的深度复制。图中的每个节点都包含一个标签（`int`)和它邻居节点的列表(`List[UndirectedGraphNode]`)。给定节点和其邻居节点之间都有一条边。

**`OJ`的无向图系列（从而我们可以了解错误的输出）：节点的标签具有唯一性。**

我们使用`#`作为每个节点之间的分隔符，`,`作为节点的标签以及节点的所有邻居。

将序列图`{0,1,2#1,2#2,2}`作为一个示例：

从上面可以看出，这个图一共有使用`#`分割的三个节点。

- 第1个节点的标签为`0`，节点`0`同时连接`1`和`2`
- 第2个节点的标签为`1`，节点`1`连接`2`
- 第3个节点的标签为`2`，节点`2`只连接它自己，形成一个环

可视化之后如下图所示：
 ```

       1
      / \
     /   \
    0 --- 2
        / \
        \_/
```
**注意**：树序列化的信息只是为了帮助我们了解错误的输出。我们不需要了解序列化来解决这个问题。